### rate: 미국 국채
### df: FOMC, Interview, Speech
### news_df: NEWS
### is_df: interview + speech

##### 뉴스 데이터 합치기

In [ ]:
import pandas as pd
import os

# 엑셀 파일 병합하기
filepath = '/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/'

# 파일 경로
file_list = os.listdir(filepath)
# 경로에 있는 모든 csv 파일 리스트 불러오기
file_list_csv = [file for file in file_list if file.endswith('.csv')]

# 저장할 dataframe
merged_df = pd.DataFrame()

for file in file_list_csv:
    # 파일의 경로
    file_path = os.path.join(filepath, file)

    try:
        # read_csv 기능으로 파일 읽음
        df = pd.read_csv(file_path, dtype='object')

        # 파일이 비어있는지 확인
        if not df.empty:
            # 병합된 데이터프레임에 추가
            merged_df = merged_df.append(df)
        else:
            print(f"Warning: File {file} is empty. Skipping...")
            # 비어있는 파일 삭제
            os.remove(file_path)
    except pd.errors.EmptyDataError:
        print(f"Error: File {file} is empty. Skipping...")
        # 비어있는 파일 삭제
        os.remove(file_path)

# 병합 엑셀 파일 저장
output_filepath = "/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/merged_news_2019~2023.csv"
merged_df.to_csv(output_filepath, index=False, encoding='utf-8-sig', mode='w')

# 병합된 데이터프레임 출력
print(merged_df.head())

# 병합된 데이터프레임의 크기 출력
print("Merged DataFrame Shape:", merged_df.shape)

In [3]:
import pandas as pd

merged_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/merged_news_2019~2023.csv")

In [4]:
# 'title' 컬럼 삭제
merged_df = merged_df.drop(columns=['Unnamed: 0','url','title', 'media'])

In [5]:
merged_df.rename(columns={'content':'Text'}, inplace = True)

In [6]:
# 'date' 열을 datetime 형식으로 변환
merged_df['date'] = pd.to_datetime(merged_df['date'])

# 'date' 열을 원하는 형식으로 변환
merged_df['date'] = merged_df['date'].dt.strftime("%Y-%m-%d")

merged_df

,date,Text
0,2019-08-26,Leading stock exchange BSE on Monday launched ...
1,2019-08-26,Leading stock exchange BSE on Monday launched ...
2,2019-11-22,New Delhi: Leading stock exchange NSE on Frida...
3,2019-03-29,The government has left unchanged the interest...
4,2019-06-28,Interest rates on small savings schemes such a...
...,...,...
395826,2023-09-21,Our portfolio is strategically focused on sect...
395827,2023-03-01,"In this episode of ""Intelligence Matters,"" hos..."
395828,2023-03-01,"In this episode of ""Intelligence Matters,"" hos..."
395829,2023-06-01,With help from Eli Okun and Garrett Ross Kevin...


##### FOMC, Interview, Speech

In [4]:
import pandas as pd

# 주어진 텍스트 파일 경로
file_path = '/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/merge/merged_2018~2023.txt'

# 각 항목을 담을 리스트
data_list = []

# 텍스트 파일을 읽어서 각 항목의 시작 지점을 찾아 리스트에 추가
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        # 예시로 각 행이 개별 문장 또는 단락으로 구분되어 있다고 가정하고 '\n'로 나누기
        parts = line.strip().split('\n')

        # 리스트에 추가
        data_list.extend(parts)

# 데이터프레임으로 변환
df = pd.DataFrame(data_list, columns=['Text'])
df

,Text
0,"﻿""STEVE LIESMAN. Mr. Chairman right, the micro..."
1,"""JIM TANKERSLEY. Hi, Mr. Chairman. Jim Tankers..."
2,"""NICK TIMIRAOS. Thank you. Nick Timiraos, the ..."
3,"""SAM FLEMING. Thanks very much. Sam Fleming fr..."
4,"""HOWARD SCHNEIDER. Howard Schneider with Reute..."
...,...
128,"""Minutes of the Federal Open Market Committee ..."
129,"""Minutes of the Federal Open Market Committee ..."
130,"""Minutes of the Federal Open Market Committee ..."
131,"""Minutes of the Federal Open Market Committee ..."


##### news, interview+speech, fomc

In [8]:
merged_df = merged_df.sort_values(by='date')
merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/sort_merged_news_2019~2023.csv')

In [16]:
# news
news_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/sort_merged_news_2019~2023.csv")

In [ ]:
import pandas as pd

# 'Text' 열의 부동 소수점 값을 처리하기 위해 'Text' 열을 문자열로 변환합니다
news_df['Text'] = news_df['Text'].astype(str)

# 'date'로 그룹화하고 텍스트를 결합
news_df = news_df.groupby('date')['Text'].apply(lambda x: ' '.join(x)).reset_index()

news_df

In [ ]:
news_df['Text'][0]

In [ ]:
# interview + speech
is_df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/연설+인터뷰.xlsx")
is_df.rename(columns={'Unnamed: 0':'date', 'Unnamed: 1':'Text'}, inplace = True)
is_df.sort_values(by = 'date', inplace = True)
is_df = is_df.groupby('date')['Text'].apply(lambda x: ' '.join(x)).reset_index()
is_df

In [ ]:
# fomc
fomc_df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/fomc회의록 merged.xlsx")
fomc_df

In [2]:
# news_df + is_df + fomc_df
import pandas as pd
merged_df = pd.merge(is_df, news_df, on='date', how = 'outer')

merged_df
merged_df['Text_x'].fillna('', inplace=True)
merged_df['Text_y'].fillna('', inplace=True)
merged_df['Text'] = merged_df['Text_x'] + ' ' + merged_df['Text_y']

# 결과 확인
print(merged_df[['date', 'Text']])

NameError: ignored

In [27]:
# 데이터프레임 병합
merged_df = pd.merge(is_df, news_df, on='date', how='outer')

# 열 이름 확인
# print(merged_df.columns)

# 열 이름에 따라 그룹화 및 문자열 결합
grouped_data = merged_df.groupby('date')['Text_x'].apply(lambda x: ' '.join(str(item) for item in x if pd.notna(item))).reset_index()

# 결과 확인
grouped_data

,date,Text_x
0,2018.03.21,"I have a brief statement, and then I'll be hap..."
1,2018.06.13,"JIM TANKERSLEY. Hi, Mr. Chairman. Jim Tankersl..."
2,2018.09.26,"NICK TIMIRAOS. Thank you. Nick Timiraos, the W..."
3,2018.12.19,Over the past year the economy has been growin...
4,2019-01-01,
...,...,...
1822,2023.05.03,I didn't-I didn't take part in creating the re...
1823,2023.06.14,with where it is could you also talk briefly a...
1824,2023.07.26,My colleagues and I remain squarely focused on...
1825,2023.09.20,"about how tight the economy is, and hence don'..."


In [ ]:
# 전처리


##### 금리

In [38]:
rate = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Unclock-watchers/news/미국 1개월 채권수익률 과거 데이터 (1).csv')
rate.rename(columns={'날짜':'date', '종가':'closing price', '변동 %':'fluctuation'}, inplace = True)
rate = rate.drop(columns=['시가', '고가', '저가'])

In [41]:
rate.sort_values(by = 'date')

,date,closing price,fluctuation
1511,2018- 01- 01,1.263,0.18%
1510,2018- 01- 02,1.240,-1.84%
1509,2018- 01- 03,1.299,4.76%
1508,2018- 01- 04,1.269,-2.31%
1507,2018- 01- 05,1.256,-1.02%
...,...,...,...
4,2023- 10- 25,5.399,-0.06%
3,2023- 10- 26,5.395,-0.07%
2,2023- 10- 27,5.408,0.24%
1,2023- 10- 30,5.400,-0.15%


##### 금리 + 전처리 된 데이터